In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import KFold
import pathlib

In [ ]:

with open('file.json', 'r') as file:
    data = json.load(file)

In [ ]:

ds = pd.read_csv("C:\\Users\\Equipo\\Desktop\\NuevoVacas\\Train\\training_ds.csv")#importar csv con el dataset como ds
ds.head()


In [ ]:
train, test = train_test_split(ds, test_size=0.25)

In [ ]:
train_ds = train
test_ds = test


In [ ]:
(x_train, y_train) = train_ds.filename, train_ds.label
(x_test, y_test) = test_ds.filename, test_ds.label

In [ ]:
train_ds

In [ ]:
filepath = "C:\\Users\\Equipo\\Desktop\\NuevoVacas\\"

In [ ]:
data_dir = tf.keras.utils.get_file(origin=filepath, fname='dataset')

In [ ]:

(x_train, y_train), (x_test, y_test) = ds
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


In [ ]:

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [ ]:

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


model = MyModel()


In [ ]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='test_accuracy')



In [ ]:

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)



In [ ]:

EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

In [ ]:
FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})
